# Create global summary and home README

##  Configuration

In [ ]:
# Import dependencies
import os
import glob
import pandas as pd
import contextily as ctx
import geopandas as gpd
import matplotlib.pyplot as plt

## Generate global summaries

In [ ]:
# ------------------------------------------------------------------------------
#  earthquake_information summary
# ------------------------------------------------------------------------------
events = sorted(glob.glob(os.path.join('..', '**','earthquake_information.csv'), 
                recursive=True))
eqi = []
print('Generating `earthquake_information` summary file')
for file_path in events:
    # print(file_path)
    df = pd.read_csv(file_path)
    event_name = file_path.split(os.sep)[-2]
    
    if event_name != 'Event_Template':
        df.loc[-1, :] = 'Event_ID', event_name
        df.sort_index(inplace=True)
        
        df = df.set_index('FIELD').transpose()
        eqi.append(df)

eq_info = pd.concat(eqi, ignore_index=True)
eq_info.sort_values('Year', inplace=True, ascending=False)

# Save global suymmary
save_as = os.path.join('..', 'World', 'world_eq_info.csv')
eq_info.to_csv(save_as, index=False, encoding='utf-8')
print(f'   File saved as {save_as}')

# ------------------------------------------------------------------------------
#  Impact_All_ID_0 summary
# ------------------------------------------------------------------------------
files = sorted(glob.glob(os.path.join('..', '**', 'Impact_All_ID_0.csv'),
                        recursive=True))

impact = []
print('Generating `Impact_All_ID_0` summary file')
for file_path in files:
    # print(file_path)
    event_name = file_path.split(os.sep)[2]
    try:
        df = pd.read_csv(file_path)
        df.insert(0, 'EVENT', event_name)
        impact.append(df)   
    except UnicodeDecodeError as e:
        print(e)
        print(event_name)

global_impact = pd.concat(impact, axis=0)

# Save global suymmary
save_as = os.path.join('..', 'World', 'world_impact_ID_0.csv')
global_impact.to_csv(save_as, index=False, encoding='utf-8')
print(f'   File saved as {save_as}')


## Home README

### Create table of events

In [ ]:
df = pd.read_csv(os.path.join('..', 'World', 'world_eq_info.csv'))

# Define columns to include in Home README
cols = ['Country', 'Year', 'Event_ID', 'Event_Name', 'Mw', 'Depth_(km)', 'Max_Intensity_(MMI)']
table = df.loc[~df.Mw.isna(), cols].copy()
table.sort_values(by=['Country', 'Year'], inplace=True)
table.reset_index(inplace=True, drop=True)

# Add links to events page

table['Event_Name'] = '[' + table.Event_Name+ '](./' + table.Country + '/' + table.Event_ID + ')'
table.drop(columns='Event_ID', inplace=True)

# Store table in markdown
events_table = table.to_markdown(index=True)

table

### Figure with event coverage

In [ ]:
# Create GeoDataFrame
gdf = gpd.GeoDataFrame(df,
                       geometry=gpd.points_from_xy(df.Longitude, 
                                                   df.Latitude),
                       crs='EPSG:4326')

# Plot events
fig, ax = plt.subplots(figsize=(12, 10)) # , alpha=0.5) # alpha not supported in OQ libraries
ax.set_aspect('equal')
gdf.plot(ax=ax,
         column='Event_Name',
         alpha=0.6,
         # color='orange',
         marker='o', 
         markersize=180,
         edgecolor='grey', 
         linewidth=0.5,
         label='Event_Name')

# Add labels to figure !!!! Needs more work because of overlaping labels
# lbl = 'Year' # options: 'Year', 'Event_Name '
# for x, y, label in zip(gdf.geometry.x, gdf.geometry.y, gdf[lbl]):
#     ax.annotate(label, 
#                 xy=(x, y), 
#                 xytext=(3, 3), 
#                 textcoords="offset points",
#                 color='black',
#                 fontsize='small')

# Add default basemap: Stamen Terrain (uses Web Mercator --> epsg=3857)
# Additional basemaps available at:
# https://contextily.readthedocs.io/en/latest/providers_deepdive.html#What-is-this-%E2%80%9Cprovider%E2%80%9D-object-?
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite,
                crs=gdf.crs, alpha=0.8)


# Save figure
file_path = os.path.join('..', 'World', 'eq_events.png')
fig.savefig(file_path, facecolor="w", 
            dpi=300, bbox_inches='tight')

### Text of home README

In [ ]:
# README text

text = f'''
<div align='center'>
<p align="center">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/5b/Global_Earthquake_Model_Logo.png/440px-Global_Earthquake_Model_Logo.png" alt="GEM Foundation" width="300"/>
</p>
<a href='./earthquake_scenarios/'>
    <img src='https://img.shields.io/badge/Earthquake_Scenarios-green?style=for-the-badge'>
</a>
<a href='./World/'>
    <img src='https://img.shields.io/badge/Global_coverage-gray?style=for-the-badge'>
</a>
<a href='./contribute_guidelines.md'>
    <img src='https://img.shields.io/badge/Contribute-orange?style=for-the-badge'>
</a>
<a href='LICENSE.txt'>
    <img src='https://img.shields.io/badge/LICENSE-blue?style=for-the-badge'>
</a>
</div>

# 🔎 GEM Earthquake Scenario Database

[[_TOC_]]

# ✨ Overview

> The v2023.0.0 release for the GEM's Earthquake Scenario Database is available! 🥳 🚀

This repository is a collection of earthquake footprints and consequences from past events.

<div align='left'>
    <img src="./World/eq_events.png" alt="GEM's ECD events" width="700"/>
</div>

Find the global summary impact data in the [World folder](./World).

## 🗺️ Database coverage

The following events are available in the database.

<details>
<summary> Table with available events
</summary>

{events_table}

</details>

# 🚀 Model versions

Each version of the archive that is released can be accessed by changing from the `main` branch to the `tag` of a given version.
The `main` branch could contain the work-in-progress of the next version of the model.

| Version   | Release Notes                                                            |
|-----------|--------------------------------------------------------------------------|
| [v2023.0.0](https://github.com/gem/ecd/tree/v2023.0.0) | First release with 100 earthquake scenario events.|

# 🌟 Contributors

The authors are grateful for the input from dozens of collaborators. 

# License
[![CC BY-NC-SA 4.0][cc-by-nc-sa-shield]][cc-by-nc-sa]

This work is licensed under a
[Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License][cc-by-nc-sa].

[![CC BY-NC-SA 4.0][cc-by-nc-sa-image]][cc-by-nc-sa]

[cc-by-nc-sa]: http://creativecommons.org/licenses/by-nc-sa/4.0/
[cc-by-nc-sa-image]: https://licensebuttons.net/l/by-nc-sa/4.0/88x31.png
[cc-by-nc-sa-shield]: https://img.shields.io/badge/License-CC%20BY--NC--SA%204.0-lightgrey.svg

# 🤔 Frequently asked questions

### How to contribute?

You can follow the instructions indicated in the [contributing guidelines](./contribute_guidelines.md).

### Which version am I seeing? How to change the version?

By default, you will see the files in the repository in the `main` branch. Each version of the model that is released can be accessed is marked with a `tag`. By changing the tag version at the top of the repository, you can change see the files for a given version.

Note that the `main` branch could contain the work-in-progress of the next version of the model.

### How do I download the data for a given version?

For each version, a related zip file is available in the [release section](https://github.com/gem/global_exposure_model/releases).

'''

# Save README
f=open(os.path.join('..', 'README.md'), 'w')
f.write(text)
f.close()
